In [1]:
# from google.colab import files
# from google.colab import drive
import os
import io
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare
import numpy as np

from datetime import datetime
# drive.mount('/content/drive') #/MyDrive/Colab Notebooks/데이터/기상데이터

In [15]:
wind = pd.read_csv('../data/ta.csv', index_col = 0)
lon_lat = pd.read_csv('../data//mapping_0713.csv', index_col = 0)

wind = wind.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_ws' : 'avg_ws', 'a.mix_ws' : 'mix_ws', 'a.max_ws' : 'max_ws', 'a.max_wd' : 'max_wd', 'a.max_ws_wd' : 'max_ws_wd', 'a.max_ins_ws' : 'max_ins_ws', 'a.max_ins_ws_wd' : 'max_ins_ws_wd', 'a.mix_wd' : 'mix_wd'})

# wind = pd.merge(wind, lon_lat, on = ['stn_id'])

wind['date'] = wind['tma'].map(lambda date_wind: date_wind[:10])
wind['time'] = wind['tma'].map(lambda time_wind: time_wind[11:-2])
wind['date'] = pd.to_datetime(wind['date'], format='%Y-%m-%d')
wind['time'] = pd.to_datetime(wind['time'], format='%H:%M:%S')


wind_ds = wind.groupby(['date', 'stn_id'], as_index=False).median()

wind_ds = pd.merge(wind_ds, lon_lat, on = ['stn_id'])

del wind_ds['name']
del wind_ds['address']

wind = wind_ds

In [16]:
wind

,date,stn_id,avg_ws,mix_ws,max_ws,max_ws_wd,max_wd,max_ins_ws,max_ins_ws_wd,mix_wd,lat,long,add
0,2012-01-01,12,3.7,3.3,9.9,350.7,67.5,16.2,350.9,3.0,36.53330,126.31670,충남
1,2012-01-02,12,2.0,1.5,4.9,359.4,67.5,8.6,182.8,53.4,36.53330,126.31670,충남
2,2012-01-03,12,3.8,2.1,8.2,346.4,360.0,14.8,0.1,23.3,36.53330,126.31670,충남
3,2012-01-04,12,6.1,6.0,9.7,351.3,360.0,17.5,313.2,359.4,36.53330,126.31670,충남
4,2012-01-05,12,3.1,2.2,6.0,354.9,67.5,8.7,0.0,43.7,36.53330,126.31670,충남
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633580,2022-11-05,239,0.8,0.5,2.6,81.0,315.0,3.9,329.1,346.9,36.48522,127.24438,충남
2633581,2022-11-06,239,0.5,0.1,2.1,47.0,67.5,4.2,303.8,31.8,36.48522,127.24438,충남
2633582,2022-11-07,239,0.5,0.2,1.7,90.1,45.0,2.5,92.8,49.6,36.48522,127.24438,충남
2633583,2022-11-08,239,0.7,0.4,2.1,89.3,67.5,3.2,92.8,15.4,36.48522,127.24438,충남


# -------------------------------------------  
# Linear Interpolation

In [17]:
days = pd.DataFrame({'date' : pd.date_range(start="2012-02-01",end="2022-10-31")})

In [18]:
wind['date'] = pd.to_datetime(wind['date'])
wind['date'] = pd.to_datetime(wind['date'], format='%Y-%m-%d')


In [19]:
wind_Gangwon = wind[wind['add']=='강원']
wind_Gyeonggi = wind[wind['add']=='경기']
wind_Gyeongnam = wind[wind['add']=='경남']
wind_Gyeongbuk = wind[wind['add']=='경북']
wind_Gwangju = wind[wind['add']=='광주']
wind_Daegu = wind[wind['add']=='대구']
wind_Daejeon = wind[wind['add']=='대전']
wind_Busan = wind[wind['add']=='부산']
wind_Seoul = wind[wind['add']=='서울']
wind_Sejong = wind[wind['add']=='세종']
wind_Ulsan = wind[wind['add']=='울산']
wind_Incheon = wind[wind['add']=='인천']
wind_Jeonnam = wind[wind['add']=='전남']
wind_Jeonbuk = wind[wind['add']=='전북']
wind_Jeju = wind[wind['add']=='제주']
wind_Chungnam = wind[wind['add']=='충남']
wind_Chungbuk = wind[wind['add']=='충북']

## 강원

'a.avg_ws' : 'avg_ws', 'a.mix_ws' : 'mix_ws', 'a.max_ws' : 'max_ws', 'a.max_wd' : 'max_wd', 'a.max_ws_wd' : 'max_ws_wd', 'a.max_ins_ws' : 'max_ins_ws', 'a.max_ins_ws_wd' : 'max_ins_ws_wd', 'a.mix_wd' : 'mix_wd'

In [20]:
wind_Gangwon_outer = pd.merge(wind_Gangwon, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Gangwon_outer['avg_ws'] = wind_Gangwon_outer['avg_ws'].interpolate(method = 'values')
wind_Gangwon_outer['mix_ws'] = wind_Gangwon_outer['mix_ws'].interpolate(method = 'values')
wind_Gangwon_outer['max_ws'] = wind_Gangwon_outer['max_ws'].interpolate(method = 'values')
wind_Gangwon_outer['max_wd'] = wind_Gangwon_outer['max_wd'].interpolate(method = 'values')
wind_Gangwon_outer['max_ws_wd'] = wind_Gangwon_outer['max_ws_wd'].interpolate(method = 'values')
wind_Gangwon_outer['max_ins_ws'] = wind_Gangwon_outer['max_ins_ws'].interpolate(method = 'values')
wind_Gangwon_outer['max_ins_ws_wd'] = wind_Gangwon_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Gangwon_outer['mix_wd'] = wind_Gangwon_outer['mix_wd'].interpolate(method = 'values')

wind_Gangwon_outer['add'] = '강원'
del wind_Gangwon_outer['stn_id']
del wind_Gangwon_outer['lat']
del wind_Gangwon_outer['long']

## 경기

In [21]:
wind_Gyeonggi_outer = pd.merge(wind_Gyeonggi, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Gyeonggi_outer['avg_ws'] = wind_Gyeonggi_outer['avg_ws'].interpolate(method = 'values')
wind_Gyeonggi_outer['mix_ws'] = wind_Gyeonggi_outer['mix_ws'].interpolate(method = 'values')
wind_Gyeonggi_outer['max_ws'] = wind_Gyeonggi_outer['max_ws'].interpolate(method = 'values')
wind_Gyeonggi_outer['max_wd'] = wind_Gyeonggi_outer['max_wd'].interpolate(method = 'values')
wind_Gyeonggi_outer['max_ws_wd'] = wind_Gyeonggi_outer['max_ws_wd'].interpolate(method = 'values')
wind_Gyeonggi_outer['max_ins_ws'] = wind_Gyeonggi_outer['max_ins_ws'].interpolate(method = 'values')
wind_Gyeonggi_outer['max_ins_ws_wd'] = wind_Gyeonggi_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Gyeonggi_outer['mix_wd'] = wind_Gyeonggi_outer['mix_wd'].interpolate(method = 'values')

wind_Gyeonggi_outer['add'] = '경기'
del wind_Gyeonggi_outer['stn_id']
del wind_Gyeonggi_outer['lat']
del wind_Gyeonggi_outer['long']

## 경남

In [22]:
wind_Gyeongnam_outer = pd.merge(wind_Gyeongnam, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Gyeongnam_outer['avg_ws'] = wind_Gyeongnam_outer['avg_ws'].interpolate(method = 'values')
wind_Gyeongnam_outer['mix_ws'] = wind_Gyeongnam_outer['mix_ws'].interpolate(method = 'values')
wind_Gyeongnam_outer['max_ws'] = wind_Gyeongnam_outer['max_ws'].interpolate(method = 'values')
wind_Gyeongnam_outer['max_wd'] = wind_Gyeongnam_outer['max_wd'].interpolate(method = 'values')
wind_Gyeongnam_outer['max_ws_wd'] = wind_Gyeongnam_outer['max_ws_wd'].interpolate(method = 'values')
wind_Gyeongnam_outer['max_ins_ws'] = wind_Gyeongnam_outer['max_ins_ws'].interpolate(method = 'values')
wind_Gyeongnam_outer['max_ins_ws_wd'] = wind_Gyeongnam_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Gyeongnam_outer['mix_wd'] = wind_Gyeongnam_outer['mix_wd'].interpolate(method = 'values')


wind_Gyeongnam_outer['add'] = '경남'
del wind_Gyeongnam_outer['stn_id']
del wind_Gyeongnam_outer['lat']
del wind_Gyeongnam_outer['long']

## 경북

In [23]:
wind_Gyeongbuk_outer = pd.merge(wind_Gyeongbuk, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Gyeongbuk_outer['avg_ws'] = wind_Gyeongbuk_outer['avg_ws'].interpolate(method = 'values')
wind_Gyeongbuk_outer['mix_ws'] = wind_Gyeongbuk_outer['mix_ws'].interpolate(method = 'values')
wind_Gyeongbuk_outer['max_ws'] = wind_Gyeongbuk_outer['max_ws'].interpolate(method = 'values')
wind_Gyeongbuk_outer['max_wd'] = wind_Gyeongbuk_outer['max_wd'].interpolate(method = 'values')
wind_Gyeongbuk_outer['max_ws_wd'] = wind_Gyeongbuk_outer['max_ws_wd'].interpolate(method = 'values')
wind_Gyeongbuk_outer['max_ins_ws'] = wind_Gyeongbuk_outer['max_ins_ws'].interpolate(method = 'values')
wind_Gyeongbuk_outer['max_ins_ws_wd'] = wind_Gyeongbuk_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Gyeongbuk_outer['mix_wd'] = wind_Gyeongbuk_outer['mix_wd'].interpolate(method = 'values')


wind_Gyeongbuk_outer['add'] = '경북'
del wind_Gyeongbuk_outer['stn_id']
del wind_Gyeongbuk_outer['lat']
del wind_Gyeongbuk_outer['long']

## 광주

In [24]:
wind_Gwangju_outer = pd.merge(wind_Gwangju, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Gwangju_outer['avg_ws'] = wind_Gwangju_outer['avg_ws'].interpolate(method = 'values')
wind_Gwangju_outer['mix_ws'] = wind_Gwangju_outer['mix_ws'].interpolate(method = 'values')
wind_Gwangju_outer['max_ws'] = wind_Gwangju_outer['max_ws'].interpolate(method = 'values')
wind_Gwangju_outer['max_wd'] = wind_Gwangju_outer['max_wd'].interpolate(method = 'values')
wind_Gwangju_outer['max_ws_wd'] = wind_Gwangju_outer['max_ws_wd'].interpolate(method = 'values')
wind_Gwangju_outer['max_ins_ws'] = wind_Gwangju_outer['max_ins_ws'].interpolate(method = 'values')
wind_Gwangju_outer['max_ins_ws_wd'] = wind_Gwangju_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Gwangju_outer['mix_wd'] = wind_Gwangju_outer['mix_wd'].interpolate(method = 'values')


wind_Gwangju_outer['add'] = '광주'
del wind_Gwangju_outer['stn_id']
del wind_Gwangju_outer['lat']
del wind_Gwangju_outer['long']

## 대구

In [25]:
wind_Daegu_outer = pd.merge(wind_Daegu, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Daegu_outer['avg_ws'] = wind_Daegu_outer['avg_ws'].interpolate(method = 'values')
wind_Daegu_outer['mix_ws'] = wind_Daegu_outer['mix_ws'].interpolate(method = 'values')
wind_Daegu_outer['max_ws'] = wind_Daegu_outer['max_ws'].interpolate(method = 'values')
wind_Daegu_outer['max_wd'] = wind_Daegu_outer['max_wd'].interpolate(method = 'values')
wind_Daegu_outer['max_ws_wd'] = wind_Daegu_outer['max_ws_wd'].interpolate(method = 'values')
wind_Daegu_outer['max_ins_ws'] = wind_Daegu_outer['max_ins_ws'].interpolate(method = 'values')
wind_Daegu_outer['max_ins_ws_wd'] = wind_Daegu_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Daegu_outer['mix_wd'] = wind_Daegu_outer['mix_wd'].interpolate(method = 'values')


wind_Daegu_outer['add'] = '대구'
del wind_Daegu_outer['stn_id']
del wind_Daegu_outer['lat']
del wind_Daegu_outer['long']

## 대전

In [26]:
wind_Daejeon_outer = pd.merge(wind_Daejeon, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Daejeon_outer['avg_ws'] = wind_Daejeon_outer['avg_ws'].interpolate(method = 'values')
wind_Daejeon_outer['mix_ws'] = wind_Daejeon_outer['mix_ws'].interpolate(method = 'values')
wind_Daejeon_outer['max_ws'] = wind_Daejeon_outer['max_ws'].interpolate(method = 'values')
wind_Daejeon_outer['max_wd'] = wind_Daejeon_outer['max_wd'].interpolate(method = 'values')
wind_Daejeon_outer['max_ws_wd'] = wind_Daejeon_outer['max_ws_wd'].interpolate(method = 'values')
wind_Daejeon_outer['max_ins_ws'] = wind_Daejeon_outer['max_ins_ws'].interpolate(method = 'values')
wind_Daejeon_outer['max_ins_ws_wd'] = wind_Daejeon_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Daejeon_outer['mix_wd'] = wind_Daejeon_outer['mix_wd'].interpolate(method = 'values')

wind_Daejeon_outer['add'] = '대전'
del wind_Daejeon_outer['stn_id']
del wind_Daejeon_outer['lat']
del wind_Daejeon_outer['long']

## 부산

In [27]:
wind_Busan_outer = pd.merge(wind_Busan, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Busan_outer['avg_ws'] = wind_Busan_outer['avg_ws'].interpolate(method = 'values')
wind_Busan_outer['mix_ws'] = wind_Busan_outer['mix_ws'].interpolate(method = 'values')
wind_Busan_outer['max_ws'] = wind_Busan_outer['max_ws'].interpolate(method = 'values')
wind_Busan_outer['max_wd'] = wind_Busan_outer['max_wd'].interpolate(method = 'values')
wind_Busan_outer['max_ws_wd'] = wind_Busan_outer['max_ws_wd'].interpolate(method = 'values')
wind_Busan_outer['max_ins_ws'] = wind_Busan_outer['max_ins_ws'].interpolate(method = 'values')
wind_Busan_outer['max_ins_ws_wd'] = wind_Busan_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Busan_outer['mix_wd'] = wind_Busan_outer['mix_wd'].interpolate(method = 'values')

wind_Busan_outer['add'] = '부산'
del wind_Busan_outer['stn_id']
del wind_Busan_outer['lat']
del wind_Busan_outer['long']

## 서울

In [28]:
wind_Seoul_outer = pd.merge(wind_Seoul, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Seoul_outer['avg_ws'] = wind_Seoul_outer['avg_ws'].interpolate(method = 'values')
wind_Seoul_outer['mix_ws'] = wind_Seoul_outer['mix_ws'].interpolate(method = 'values')
wind_Seoul_outer['max_ws'] = wind_Seoul_outer['max_ws'].interpolate(method = 'values')
wind_Seoul_outer['max_wd'] = wind_Seoul_outer['max_wd'].interpolate(method = 'values')
wind_Seoul_outer['max_ws_wd'] = wind_Seoul_outer['max_ws_wd'].interpolate(method = 'values')
wind_Seoul_outer['max_ins_ws'] = wind_Seoul_outer['max_ins_ws'].interpolate(method = 'values')
wind_Seoul_outer['max_ins_ws_wd'] = wind_Seoul_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Seoul_outer['mix_wd'] = wind_Seoul_outer['mix_wd'].interpolate(method = 'values')

wind_Seoul_outer['add'] = '서울'
del wind_Seoul_outer['stn_id']
del wind_Seoul_outer['lat']
del wind_Seoul_outer['long']

## 세종

In [30]:
wind_Sejong_outer = pd.merge(wind_Sejong, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Sejong_outer['avg_ws'] = wind_Sejong_outer['avg_ws'].interpolate(method = 'values')
wind_Sejong_outer['mix_ws'] = wind_Sejong_outer['mix_ws'].interpolate(method = 'values')
wind_Sejong_outer['max_ws'] = wind_Sejong_outer['max_ws'].interpolate(method = 'values')
wind_Sejong_outer['max_wd'] = wind_Sejong_outer['max_wd'].interpolate(method = 'values')
wind_Sejong_outer['max_ws_wd'] = wind_Sejong_outer['max_ws_wd'].interpolate(method = 'values')
wind_Sejong_outer['max_ins_ws'] = wind_Sejong_outer['max_ins_ws'].interpolate(method = 'values')
wind_Sejong_outer['max_ins_ws_wd'] = wind_Sejong_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Sejong_outer['mix_wd'] = wind_Sejong_outer['mix_wd'].interpolate(method = 'values')


wind_Sejong_outer['add'] = '세종'
del wind_Sejong_outer['stn_id']
del wind_Sejong_outer['lat']
del wind_Sejong_outer['long']

## 울산

In [31]:
wind_Ulsan_outer = pd.merge(wind_Ulsan, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Ulsan_outer['avg_ws'] = wind_Ulsan_outer['avg_ws'].interpolate(method = 'values')
wind_Ulsan_outer['mix_ws'] = wind_Ulsan_outer['mix_ws'].interpolate(method = 'values')
wind_Ulsan_outer['max_ws'] = wind_Ulsan_outer['max_ws'].interpolate(method = 'values')
wind_Ulsan_outer['max_wd'] = wind_Ulsan_outer['max_wd'].interpolate(method = 'values')
wind_Ulsan_outer['max_ws_wd'] = wind_Ulsan_outer['max_ws_wd'].interpolate(method = 'values')
wind_Ulsan_outer['max_ins_ws'] = wind_Ulsan_outer['max_ins_ws'].interpolate(method = 'values')
wind_Ulsan_outer['max_ins_ws_wd'] = wind_Ulsan_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Ulsan_outer['mix_wd'] = wind_Ulsan_outer['mix_wd'].interpolate(method = 'values')

wind_Ulsan_outer['add'] = '울산'
del wind_Ulsan_outer['stn_id']
del wind_Ulsan_outer['lat']
del wind_Ulsan_outer['long']

## 인천

In [32]:
wind_Incheon_outer = pd.merge(wind_Incheon, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Incheon_outer['avg_ws'] = wind_Incheon_outer['avg_ws'].interpolate(method = 'values')
wind_Incheon_outer['mix_ws'] = wind_Incheon_outer['mix_ws'].interpolate(method = 'values')
wind_Incheon_outer['max_ws'] = wind_Incheon_outer['max_ws'].interpolate(method = 'values')
wind_Incheon_outer['max_wd'] = wind_Incheon_outer['max_wd'].interpolate(method = 'values')
wind_Incheon_outer['max_ws_wd'] = wind_Incheon_outer['max_ws_wd'].interpolate(method = 'values')
wind_Incheon_outer['max_ins_ws'] = wind_Incheon_outer['max_ins_ws'].interpolate(method = 'values')
wind_Incheon_outer['max_ins_ws_wd'] = wind_Incheon_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Incheon_outer['mix_wd'] = wind_Incheon_outer['mix_wd'].interpolate(method = 'values')

wind_Incheon_outer['add'] = '인천'
del wind_Incheon_outer['stn_id']
del wind_Incheon_outer['lat']
del wind_Incheon_outer['long']

## 전남

In [33]:
wind_Jeonnam_outer = pd.merge(wind_Jeonnam, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Jeonnam_outer['avg_ws'] = wind_Jeonnam_outer['avg_ws'].interpolate(method = 'values')
wind_Jeonnam_outer['mix_ws'] = wind_Jeonnam_outer['mix_ws'].interpolate(method = 'values')
wind_Jeonnam_outer['max_ws'] = wind_Jeonnam_outer['max_ws'].interpolate(method = 'values')
wind_Jeonnam_outer['max_wd'] = wind_Jeonnam_outer['max_wd'].interpolate(method = 'values')
wind_Jeonnam_outer['max_ws_wd'] = wind_Jeonnam_outer['max_ws_wd'].interpolate(method = 'values')
wind_Jeonnam_outer['max_ins_ws'] = wind_Jeonnam_outer['max_ins_ws'].interpolate(method = 'values')
wind_Jeonnam_outer['max_ins_ws_wd'] = wind_Jeonnam_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Jeonnam_outer['mix_wd'] = wind_Jeonnam_outer['mix_wd'].interpolate(method = 'values')

wind_Jeonnam_outer['add'] = '전남'
del wind_Jeonnam_outer['stn_id']
del wind_Jeonnam_outer['lat']
del wind_Jeonnam_outer['long']

## 전북

In [34]:
wind_Jeonbuk_outer = pd.merge(wind_Jeonbuk, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Jeonbuk_outer['avg_ws'] = wind_Jeonbuk_outer['avg_ws'].interpolate(method = 'values')
wind_Jeonbuk_outer['mix_ws'] = wind_Jeonbuk_outer['mix_ws'].interpolate(method = 'values')
wind_Jeonbuk_outer['max_ws'] = wind_Jeonbuk_outer['max_ws'].interpolate(method = 'values')
wind_Jeonbuk_outer['max_wd'] = wind_Jeonbuk_outer['max_wd'].interpolate(method = 'values')
wind_Jeonbuk_outer['max_ws_wd'] = wind_Jeonbuk_outer['max_ws_wd'].interpolate(method = 'values')
wind_Jeonbuk_outer['max_ins_ws'] = wind_Jeonbuk_outer['max_ins_ws'].interpolate(method = 'values')
wind_Jeonbuk_outer['max_ins_ws_wd'] = wind_Jeonbuk_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Jeonbuk_outer['mix_wd'] = wind_Jeonbuk_outer['mix_wd'].interpolate(method = 'values')

wind_Jeonbuk_outer['add'] = '전북'
del wind_Jeonbuk_outer['stn_id']
del wind_Jeonbuk_outer['lat']
del wind_Jeonbuk_outer['long']

## 제주

In [35]:
wind_Jeju_outer = pd.merge(wind_Jeju, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Jeju_outer['avg_ws'] = wind_Jeju_outer['avg_ws'].interpolate(method = 'values')
wind_Jeju_outer['mix_ws'] = wind_Jeju_outer['mix_ws'].interpolate(method = 'values')
wind_Jeju_outer['max_ws'] = wind_Jeju_outer['max_ws'].interpolate(method = 'values')
wind_Jeju_outer['max_wd'] = wind_Jeju_outer['max_wd'].interpolate(method = 'values')
wind_Jeju_outer['max_ws_wd'] = wind_Jeju_outer['max_ws_wd'].interpolate(method = 'values')
wind_Jeju_outer['max_ins_ws'] = wind_Jeju_outer['max_ins_ws'].interpolate(method = 'values')
wind_Jeju_outer['max_ins_ws_wd'] = wind_Jeju_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Jeju_outer['mix_wd'] = wind_Jeju_outer['mix_wd'].interpolate(method = 'values')

wind_Jeju_outer['add'] = '제주'
del wind_Jeju_outer['stn_id']
del wind_Jeju_outer['lat']
del wind_Jeju_outer['long']

## 충남

In [36]:
wind_Chungnam_outer = pd.merge(wind_Chungnam, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Chungnam_outer['avg_ws'] = wind_Chungnam_outer['avg_ws'].interpolate(method = 'values')
wind_Chungnam_outer['mix_ws'] = wind_Chungnam_outer['mix_ws'].interpolate(method = 'values')
wind_Chungnam_outer['max_ws'] = wind_Chungnam_outer['max_ws'].interpolate(method = 'values')
wind_Chungnam_outer['max_wd'] = wind_Chungnam_outer['max_wd'].interpolate(method = 'values')
wind_Chungnam_outer['max_ws_wd'] = wind_Chungnam_outer['max_ws_wd'].interpolate(method = 'values')
wind_Chungnam_outer['max_ins_ws'] = wind_Chungnam_outer['max_ins_ws'].interpolate(method = 'values')
wind_Chungnam_outer['max_ins_ws_wd'] = wind_Chungnam_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Chungnam_outer['mix_wd'] = wind_Chungnam_outer['mix_wd'].interpolate(method = 'values')

wind_Chungnam_outer['add'] = '충남'
del wind_Chungnam_outer['stn_id']
del wind_Chungnam_outer['lat']
del wind_Chungnam_outer['long']

## 충북 

In [37]:
wind_Chungbuk_outer = pd.merge(wind_Chungbuk, days, on = ['date'], how = 'outer').sort_values(by = 'date')

wind_Chungbuk_outer['avg_ws'] = wind_Chungbuk_outer['avg_ws'].interpolate(method = 'values')
wind_Chungbuk_outer['mix_ws'] = wind_Chungbuk_outer['mix_ws'].interpolate(method = 'values')
wind_Chungbuk_outer['max_ws'] = wind_Chungbuk_outer['max_ws'].interpolate(method = 'values')
wind_Chungbuk_outer['max_wd'] = wind_Chungbuk_outer['max_wd'].interpolate(method = 'values')
wind_Chungbuk_outer['max_ws_wd'] = wind_Chungbuk_outer['max_ws_wd'].interpolate(method = 'values')
wind_Chungbuk_outer['max_ins_ws'] = wind_Chungbuk_outer['max_ins_ws'].interpolate(method = 'values')
wind_Chungbuk_outer['max_ins_ws_wd'] = wind_Chungbuk_outer['max_ins_ws_wd'].interpolate(method = 'values')
wind_Chungbuk_outer['mix_wd'] = wind_Chungbuk_outer['mix_wd'].interpolate(method = 'values')

wind_Chungbuk_outer['add'] = '충북'
del wind_Chungbuk_outer['stn_id']
del wind_Chungbuk_outer['lat']
del wind_Chungbuk_outer['long']

In [38]:
wind_full = pd.concat([wind_Gangwon_outer,
           wind_Gyeonggi_outer, 
           wind_Gyeongbuk_outer, 
           wind_Gyeongnam_outer,
           wind_Gwangju_outer,
           wind_Daegu_outer,
           wind_Daejeon_outer,
           wind_Busan_outer,
           wind_Seoul_outer,
           wind_Sejong_outer,
           wind_Ulsan_outer,
           wind_Incheon_outer,
           wind_Jeonnam_outer,
           wind_Jeonbuk_outer,
           wind_Jeju_outer,
           wind_Chungnam_outer,
           wind_Chungbuk_outer], axis=0)
# 날짜별 - 시도별로 정렬되어 있어야 함
wind_full.sort_values(by = ['date','add'])
wind_full = wind_full.groupby(['date', 'add'], as_index=False).median()
wind_full = wind_full.sort_values(by = ['date','add'])
# 66742 행이 있어야 해

In [40]:
wind_full = wind_full.iloc[:67269]

In [41]:
wind_full.to_csv('../data/wind_for_merge.csv')

In [42]:
pd.read_csv('../data/wind_for_merge.csv', index_col = 0)

,date,add,avg_ws,mix_ws,max_ws,max_ws_wd,max_wd,max_ins_ws,max_ins_ws_wd,mix_wd
0,2012-01-01,강원,1.80,1.30,5.100,258.50,247.50,9.00,272.800,263.90
1,2012-01-01,경기,1.50,1.30,5.700,288.20,292.50,10.60,281.300,300.60
2,2012-01-01,경남,2.10,1.55,6.600,295.10,270.00,11.30,295.300,292.50
3,2012-01-01,경북,2.40,2.00,6.700,302.45,292.50,11.30,293.500,294.45
4,2012-01-01,광주,2.10,1.70,6.400,308.65,315.00,10.40,302.350,326.65
...,...,...,...,...,...,...,...,...,...,...
67264,2022-10-31,전남,1.60,1.00,4.400,89.00,67.50,7.30,106.800,52.60
67265,2022-10-31,전북,1.20,0.50,3.800,280.20,135.00,6.10,275.600,264.70
67266,2022-10-31,제주,3.95,3.60,6.250,61.20,67.50,10.90,63.750,63.00
67267,2022-10-31,충남,1.00,0.40,3.625,178.40,135.00,5.70,193.475,169.80
